In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import progressbar as progressbar

%load_ext autoreload
%autoreload 2

In [4]:
print(f'TF version: {tf.__version__}')
# print(tf.config.list_physical_devices())

TF version: 2.0.0


# Read Data

In [5]:
# path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '../'

## Amazon Fashion

In [6]:
data_path = 'Data/Amazon/'
# file_name = 'Amazon_full' # file_name = 'Amazon_05_users' 
# file_name = 'Amazon_01_users'
# file_name = 'am_80k_users'
file_name = 'am_40k_users'
# file_name = 'am_like_ml'

## MovieLens

In [9]:
# data_path = 'data/ML/'
file_name = 'ml_1m'
# file_name = 'ML_full' # file_name = 'ML_05_users'
# file_name = 'ML_01_users'

In [10]:
df = pd.read_pickle(path + data_path + file_name)
df.head()

,user,item,datetime,rating,user_id,item_id
1845392,AGYA6NBNU1VUH,B0020B4EYU,2015-04-14,3.0,34371,11002
919114,AGYA6NBNU1VUH,B0000ZFDD4,2015-04-14,3.0,34371,179
5872323,AGYA6NBNU1VUH,B00KR6I6IK,2015-04-14,5.0,34371,75683
1237107,AGYA6NBNU1VUH,B0000ZFDD4,2015-04-14,3.0,34371,179
9971633,AGYA6NBNU1VUH,B008GXBRNM,2015-12-20,5.0,34371,35964


In [12]:
from Data_prep import leave_last_x_out_2

test_set_cfrnn = pd.read_pickle(path + 'test_set_ml_cfrnn')
val_set_cfrnn = pd.read_pickle(path  + 'val_set_ml_cfrnn')
train_set_cfrnn = pd.read_pickle(path + 'train_set_ml_cfrnn')

add_to_train, val_set = leave_last_x_out_2(val_set_cfrnn)
add_to_train_2, test_set = leave_last_x_out_2(test_set_cfrnn)

train_set = pd.concat([train_set_cfrnn, add_to_train, add_to_train_2])

In [13]:
test_set_mf = pd.read_pickle(path + data_path + file_name + '_test_mf')
val_set_mf = pd.read_pickle(path + data_path + file_name + '_val_mf')
train_set_mf = pd.read_pickle(path + data_path + file_name + '_train_mf')

In [7]:
test_set = pd.DataFrame()
train_set_og = pd.DataFrame()
pbar = progressbar.ProgressBar()
for u in pbar(df.user_id.unique()):
    last_item = df[df.user_id==u].iloc[-1:]
    test_set = test_set.append(last_item)
    remaining = df[df.user_id==u].iloc[:-1]
    train_set_og = pd.concat([train_set_og, remaining])

100% |########################################################################|


In [8]:
train_set_og

,user_id,user,item,datetime,rating,item_id
0,0,A0435554Z2P98AIGLNCS,B001MWSW8W,2015-03-17,5.0,8768
1,0,A0435554Z2P98AIGLNCS,B00KS6N9BI,2015-03-17,5.0,76377
2,0,A0435554Z2P98AIGLNCS,B007MM9VSG,2015-03-17,5.0,31002
3,0,A0435554Z2P98AIGLNCS,B00KS6NJCW,2015-03-17,1.0,76380
4,0,A0435554Z2P98AIGLNCS,B000X25EF6,2015-03-17,5.0,4730
...,...,...,...,...,...,...
371497,6039,AZZXCFBNEWIBQ,B00RL6T3FE,2017-06-22,5.0,95419
371498,6039,AZZXCFBNEWIBQ,B008UG5970,2017-08-27,4.0,34999
371499,6039,AZZXCFBNEWIBQ,B00S0VTSYA,2017-09-07,5.0,96119
371500,6039,AZZXCFBNEWIBQ,B00DVH1TP4,2017-09-07,5.0,52179


In [9]:
val_set = pd.DataFrame()
train_set = pd.DataFrame()
pbar = progressbar.ProgressBar()
for u in pbar(df.user_id.unique()):
    last_item = train_set_og[train_set_og.user_id==u].iloc[-1:]
    val_set = val_set.append(last_item)
    remaining = train_set_og[train_set_og.user_id==u].iloc[:-1]
    train_set = pd.concat([train_set, remaining])

100% |########################################################################|


In [10]:
val_set

,user_id,user,item,datetime,rating,item_id
60,0,A0435554Z2P98AIGLNCS,B000YXC2LI,2016-07-18,5.0,4986
108,1,A0464351OZXPUPKGI6HO,B01DKTA3N6,2016-12-21,5.0,138307
175,2,A0488385844WNV2OWO9X,B01075YZY4,2017-03-06,5.0,111762
234,3,A1005AVG5ETV5F,B002UD8JHQ,2018-04-30,5.0,12106
304,4,A101GG7X49PK0A,B00KMW2AZO,2017-08-07,5.0,75773
...,...,...,...,...,...,...
371122,6035,AZV1BS40E75V5,B00PUDMEOW,2018-05-02,5.0,91663
371190,6036,AZX2RDN9YXZAE,B0009GGUO0,2018-02-02,3.0,936
371239,6037,AZXX5N2AH6WPN,B00N9SFSO8,2018-03-12,5.0,84947
371303,6038,AZZT1ERHBSNQ8,B01CQVZ46A,2017-07-31,5.0,136397


In [15]:
train_set.to_pickle('../datasets/train_set_am_like_ml')
val_set.to_pickle('../datasets/val_set_am_like_ml')
test_set.to_pickle('../datasets/test_set_am_like_ml')

In [7]:
test_set = pd.read_pickle(path + 'ml_1m_test')
train_set = pd.read_pickle(path + 'ml_1m_train')

In [11]:
from Data_prep import leave_users_out
remaining, subset = leave_users_out(df, 40000)

In [12]:
len(subset)

351771

In [13]:
df = subset
df.user_id = df.user_id.astype('category').cat.codes
df.item_id = df.item_id.astype('category').cat.codes

---
# Data Prep

## Dataset Params

In [22]:
val_perc = test_perc = 0.1
n_last_items_val = n_last_items_test = 1

total_items = len(df.item_id.unique())
total_users = len(df.user_id.unique())
print(f'total_items: {total_items}')
print(f'total_users: {total_users}')

total_items: 139230
total_users: 40000


## Create Split

In [9]:
from Data_prep import train_val_test_split
datasets = train_val_test_split(df, val_perc, test_perc, n_last_items_val, n_last_items_test)
train_set, val_set, test_set = datasets

ValueError: Cannot pick 6040 users with 1 items
Try a smaller test and/or validation percentage of the data or less items to leave out

# Creating sampes

In [14]:
run_num = 'am_40k_nolf_8'

In [18]:
GMF_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'nolf': 8,
    'regs': [0,0],
    'epochs': 20,
    'sample_size': len(train_set_mf),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 2,
    'ckpt_dir': f'../NeuMF_storage/GMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

In [19]:
MLP_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'layers': [64,32,16,8],
    'reg_layers': [0,0,0,0],
    'epochs': 20,
    'sample_size': len(train_set_mf),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/MLP_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

In [23]:
NeuMF_params = {
    'learning_rate': 0.0005,
    'batch_size': 256,
    'layers': [64,32,16,8],
    'reg_layers': [0,0,0,0],
    'reg_mf': [0, 0],
    'nolf': 8,
    'epochs': 20,
    'sample_size': len(train_set_mf),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/NeuMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

## Init

In [24]:
from NCF import NCF
NCF = NCF(total_users, total_items, GMF_params, MLP_params, NeuMF_params)

NCF.build_GMF_model()
NCF.build_MLP_model()
NCF.build_NeuMF_model()

## Create Samples (MP)

## Random User sampling

In [25]:
import multiprocessing as mp        
def create_sample(user_items, train_users, train_items, params, num_processes):
    samples_sizes_split = np.array_split(np.array(range(params['sample_size'])),num_processes)
    args = []
    for samples_size in samples_sizes_split:
        args.append((user_items, train_users, train_items, len(samples_size), params['num_neg']))
    with mp.Pool(processes=num_processes) as pool:
        results = pool.starmap(create_sample_worker, args)

    user_inputs, item_inputs, labels = [], [], []
    for res_epochs in results: 
        user_inputs.extend(res_epochs['u'])
        item_inputs.extend(res_epochs['i'])
        labels.extend(res_epochs['l'])

    return user_inputs, item_inputs, labels


def create_sample_worker(user_items, train_users, train_items, sample_size, num_neg):
    user_inputs, item_inputs, labels = [], [], []
    for s in range(sample_size):
        # Add positive item
        u = np.random.choice(train_users)
        u_items = user_items[u]
        i = np.random.choice(u_items)

        user_inputs.append(u)
        item_inputs.append(i)
        labels.append(1)

        # Add negative item
        for i in range(num_neg):
            j = np.random.choice(train_items)
            while j in u_items:  # neg item j cannot be in the set of pos items of user u
                j = np.random.choice(train_items)

            user_inputs.append(u)
            item_inputs.append(j)
            labels.append(0)

    return {'u':user_inputs, 'i':item_inputs, 'l':labels}

## Single user sampling

In [26]:
import multiprocessing as mp        
def create_user_sample(user_items, train_users, train_items, params, num_processes):
    users_splits = np.array_split(np.array(train_users),num_processes)
    args = []
    for user_split in users_splits:
        args.append((user_items, user_split, train_items, params['num_neg']))
    with mp.Pool(processes=num_processes) as pool:
        results = pool.starmap(create_user_sample_worker, args)

    user_inputs, item_inputs, labels = [], [], []
    for res_epochs in results: 
        user_inputs.extend(res_epochs['u'])
        item_inputs.extend(res_epochs['i'])
        labels.extend(res_epochs['l'])

    return user_inputs, item_inputs, labels


def create_user_sample_worker(user_items, train_users, train_items, num_neg):
    user_inputs, item_inputs, labels = [], [], []
    for user in train_users:
        # All positive items for this user
        u_items = user_items[user]
        
        # Per positive item, sample num_neg negative items
        for u_item in u_items:
            pos_item = np.random.choice(u_items)

            user_inputs.append(user)
            item_inputs.append(pos_item)
            labels.append(1)

            # Add negative item
            for i in range(num_neg):
                neg_item = np.random.choice(train_items)
                while neg_item in u_items:  # neg item j cannot be in the set of pos items of user u
                    neg_item = np.random.choice(train_items)

                user_inputs.append(user)
                item_inputs.append(neg_item)
                labels.append(0)

    return {'u':user_inputs, 'i':item_inputs, 'l':labels}

In [28]:
user_items = train_set_mf.groupby('user_id')['item_id'].apply(list)
train_users = train_set_mf.user_id.unique()
train_items = train_set_mf.item_id.unique()

In [34]:
store_path = path + data_path + 'Samples/' + file_name + '_samples_' + GMF_params['num_neg'] + '_neg'
sample_name = file_name + '_sample'

In [37]:
%%time
import csv
num_processes = mp.cpu_count()
val_metrics = []
for epoch in range(30):
    print(f'Epoch: {epoch}')
    user_inputs, item_inputs, labels = create_user_sample(user_items, train_users, train_items, NCF.GMF_params, num_processes)
    samples = [user_inputs, item_inputs, labels]
    file = open(f'{store_path}/{sample_name}_{epoch}.csv', 'w+', newline='')
    with file:
        write = csv.writer(file)
        write.writerows(samples)

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
CPU times: user 1min 51s, sys: 11 s, total: 2min 2s
Wall time: 6min 21s


In [27]:
file_name

'ml_1m'

In [24]:
len(samples[0])

4993545

In [18]:
users_split = np.array_split(np.array(train_users),num_processes)

(3334,)

In [43]:
import csv

file = open(f'../NeuMF_storage/samples/ml_1m_sample_test.csv', 'w+', newline='')

with file:
    write = csv.writer(file)
    write.writerows(samples)

## Read Sample

In [16]:
%%time
with open('samples.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    samples = list(csv_reader)

CPU times: user 965 ms, sys: 290 ms, total: 1.26 s
Wall time: 1.31 s
